In [1]:
import os
import h5py
import numpy as np
from k_filters.ensrf import *
from k_filters.estkf import *


In [2]:
###LOAD TEST DATA

filename = "sample_harvey_state3.hdf5"
f = h5py.File(filename, "r")
print(f.keys())

obs =  np.array(f.get(list(f.keys())[1]))
zeta = np.array(f.get(list(f.keys())[4]))

print("wse dim = {} \n ".format(zeta.shape), 
      "obs dim = {}".format(obs.shape)
     )

<KeysViewHDF5 ['coords', 'observations', 'station_coords', 'station_node_inds', 'zeta']>
wse dim = (2, 3352598) 
  obs dim = (11,)


In [3]:
xf = zeta.T
Ne = xf.shape[1]
sigma = 0.5

Xf = xf
R = sigma * np.ones(obs.size)
Y = obs
H = np.eye(obs.size)  
HXf = H[:,::6]

params = [Xf,HXf, Y, R]

In [4]:
filter_ensrf = EnSRF(*params)
filter_estkf = ESTKF(*params)

In [5]:
xa1 = filter_ensrf._assimilate()
xa2 = filter_estkf._assimilate()

In [6]:

mean1=np.mean(xa1,axis=1)
cov1=np.cov(xa1[:1000,:],ddof=1)

mean2=np.mean(xa2,axis=1)
cov2=np.cov(xa2[:1000,:],ddof=1)

print(
    'Square Root Filter: \n'
    'xa dim = {}\n'.format(xa1.shape),
    'mean = {}\n'.format(mean1),
    'cov = {}\n'.format(cov1)
     )

print(
    'Error Subspace Filter: \n'
    'xa dim = {}\n'.format(xa2.shape),
    'mean = {}\n'.format(mean2),
    'cov = {}\n'.format(cov2)
     )

Square Root Filter: 
xa dim = (3352598, 2)
 mean = [0.54154072+0.j 0.54413802+0.j 0.55966858+0.j ... 0.30275894+0.j
 0.30275792+0.j 0.30275961+0.j]
 cov = [[ 1.62669040e-05+0.j  1.33313593e-05+0.j  9.91098353e-06+0.j ...
  -9.89622554e-06+0.j  1.01836389e-05+0.j -1.56405570e-06+0.j]
 [ 1.33313593e-05+0.j  1.09255664e-05+0.j  8.12243573e-06+0.j ...
  -8.11034098e-06+0.j  8.34588736e-06+0.j -1.28180436e-06+0.j]
 [ 9.91098353e-06+0.j  8.12243573e-06+0.j  6.03849354e-06+0.j ...
  -6.02950189e-06+0.j  6.20461504e-06+0.j -9.52936728e-07+0.j]
 ...
 [-9.89622554e-06+0.j -8.11034098e-06+0.j -6.02950189e-06+0.j ...
   6.02052363e-06+0.j -6.19537603e-06+0.j  9.51517753e-07+0.j]
 [ 1.01836389e-05+0.j  8.34588736e-06+0.j  6.20461504e-06+0.j ...
  -6.19537603e-06+0.j  6.37530661e-06+0.j -9.79152419e-07+0.j]
 [-1.56405570e-06+0.j -1.28180436e-06+0.j -9.52936728e-07+0.j ...
   9.51517753e-07+0.j -9.79152419e-07+0.j  1.50383271e-07+0.j]]

Error Subspace Filter: 
xa dim = (3352598, 2)
 mean = [0.5415407